# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW04] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier, Pool
import typing
import math

In [3]:
def gini(x: np.ndarray):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return sum(proba * (1 - proba))


def entropy(x: np.ndarray):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -sum(proba * np.log2(proba))


def gain(left_y: np.ndarray, right_y: np.ndarray, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
def out_of_bag(size, ids: np.ndarray) -> np.ndarray:
    used = set()
    for i in ids:
        used.add(i)
    return np.array([i for i in range(size) if i not in used])


def bagging(size: int) -> np.ndarray:
    ids = []
    for _ in range(size):
        ids.append(np.random.randint(size))
    return np.array(ids)


class DecisionTreeLeaf:
    def __init__(self, ys: np.ndarray):
        occur = dict()
        if ys.size > 0:
            for y in ys:
                if y in occur:
                    occur[y] += 1
                else:
                    occur[y] = 1
        for k in occur.keys():
            occur[k] /= ys.shape[0]
        self.probabilities = occur
        self.y = max(self.probabilities.keys(), key=lambda label: self.probabilities[label])
        self.size = ys.size


class DecisionTreeNode:
    def __init__(self, split_dim, left, right):
        self.split_dim = split_dim
        self.left = left
        self.right = right


class DecisionTree:
    def __init__(
            self,
            xs: np.ndarray,
            ys: np.ndarray,
            criterion: str = 'gini',
            max_depth=None,
            min_samples_leaf: int = 1,
            max_features='auto'
    ):
        self.criterion = gini if criterion == 'gini' else entropy
        self.max_depth = max_depth if max_depth else math.inf
        self.min_samples_leaf = min_samples_leaf
        self.max_feature = min(int(max_features), xs.shape[1]) if max_features != 'auto' \
            else math.ceil(xs.shape[1] ** 0.5)
        self.xs = xs
        self.ys = ys
        self.bag = bagging(len(xs))
        self.out_of_bag = out_of_bag(len(xs), self.bag)
        self.root = self.build_tree(self.xs[self.bag], self.ys[self.bag])

    def build_tree(self, xs: np.ndarray, ys: np.ndarray, depth: int = 0):
        if np.unique(np.sort(ys)).size == 1 or depth == self.max_depth:
            return DecisionTreeLeaf(ys)

        best_dim = -1
        best_ig = 0
        ids = [i for i in range(xs.shape[0])]

        dims = np.array([i for i in range(xs.shape[1])])
        np.random.shuffle(dims)
        dims = dims[:self.max_feature]

        for dim in dims:

            left_y = np.array([ys[ids[q]] for q in ids if xs[ids[q]][dim] == 0])
            right_y = np.array([ys[ids[q]] for q in ids if xs[ids[q]][dim] == 1])

            if left_y.shape[0] < self.min_samples_leaf or right_y.shape[0] < self.min_samples_leaf:
                continue

            ig = gain(left_y, right_y, self.criterion)

            if best_dim == -1 or ig > best_ig:
                best_ig = ig
                best_dim = dim

        if best_dim == -1:
            return DecisionTreeLeaf(ys)

        left_xs = np.array([xs[ids[q]] for q in ids if xs[ids[q]][best_dim] == 0])
        left_ys = np.array([ys[ids[q]] for q in ids if xs[ids[q]][best_dim] == 0])

        right_xs = np.array([xs[ids[q]] for q in ids if xs[ids[q]][best_dim] == 1])
        right_ys = np.array([ys[ids[q]] for q in ids if xs[ids[q]][best_dim] == 1])

        left_son = self.build_tree(left_xs, left_ys, depth + 1)
        right_son = self.build_tree(right_xs, right_ys, depth + 1)

        return DecisionTreeNode(best_dim, left_son, right_son)

    def get_probabilities(self, x: np.ndarray, node) -> dict:
        if isinstance(node, DecisionTreeLeaf):
            return node.probabilities
        if x[node.split_dim] == 0:
            return self.get_probabilities(x, node.left)
        else:
            return self.get_probabilities(x, node.right)

    def predict_probabilities(self, xs: np.ndarray) -> typing.List[dict]:
        return [self.get_probabilities(x, self.root) for x in xs]

    def predict(self, xs: np.ndarray) -> np.ndarray:
        probabilities = self.predict_probabilities(xs)
        return np.array([max(p.keys(), key=lambda k: p[k]) for p in probabilities])

    def out_of_bag_error(self):
        xs = self.xs[self.out_of_bag]
        ys = self.ys[self.out_of_bag]
        y_pred = self.predict(xs)
        err = sum(1 for i, y in enumerate(ys) if y != y_pred[i])
        errors = []
        for j in range(xs.shape[1]):
            xsj = xs.copy()
            np.random.shuffle(xsj[:, j])
            y_pred = self.predict(xsj)
            err_j = sum(1 for i, y in enumerate(ys) if y != y_pred[i])
            errors.append((err_j - err) / len(ys))
        return np.array(errors)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
def majority(a: np.ndarray):
    count = dict()
    for x in a:
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
    best_key = -1
    best_value = -1
    for key, value in count.items():
        if value > best_value:
            best_value = value
            best_key = key
    return best_key


class RandomForestClassifier:
    def __init__(
            self,
            criterion: str = 'gini',
            max_depth: int = None,
            min_samples_leaf: int = 1,
            max_features='auto',
            n_estimators: int = 10
    ):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.forest: typing.List[DecisionTree] = []

        self.xs = None
        self.ys = None

    def fit(self, xs: np.ndarray, ys: np.ndarray):
        self.xs = xs
        self.ys = ys
        self.forest.clear()
        for _ in range(self.n_estimators):
            self.forest.append(
                DecisionTree(
                    xs,
                    ys,
                    self.criterion,
                    self.max_depth,
                    self.min_samples_leaf,
                    self.max_features
                )
            )

    def predict(self, xs: np.ndarray) -> np.ndarray:
        votes = np.array([tree.predict(xs) for tree in self.forest])
        return np.array([majority(votes[:, i]) for i in range(votes.shape[1])])

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def feature_importance(rfc: RandomForestClassifier):
    return sum(tree.out_of_bag_error() for tree in rfc.forest) / rfc.n_estimators


def most_important_features(importance, names, k: int = 20):
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [8]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3,
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [ 1.97828809e-03 -9.89732084e-05  1.95766635e-01  1.97053012e-01
  3.87500271e-01  8.72939269e-04]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [15]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

def train_test_split(
        xs: np.ndarray,
        ys1: np.ndarray,
        ys2: np.ndarray,
        train_size: float,
) -> typing.Tuple[typing.List, typing.List, typing.List, typing.List, typing.List, typing.List]:
    train_size = int(train_size * len(xs))
    return xs[:train_size], xs[train_size:], ys1[:train_size], ys1[train_size:], ys2[:train_size], ys2[train_size:]

In [9]:
X, y_age, y_sex, features = read_dataset('src/resources/vk.csv')
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7276166456494325
Most important features:


#### Пол

In [22]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8423707440100883
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. 9o_6o_9o
6. cook_good
7. zerofat
8. igm
9. i_d_t
10. be.beauty
11. be.women
12. beauty
13. woman.blog
14. femalemem
15. 4ch
16. thesmolny
17. reflexia_our_feelings
18. bon
19. bot_maxim
20. cook_bon


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [27]:
train_data, label_values = synthetic_dataset(1000)

model = CatBoostClassifier(
    loss_function='MultiClass'
)
model.fit(
    train_data,
    label_values, 
    verbose=False
)
print("Accuracy:", np.mean(model.predict(train_data).reshape(-1) == label_values))
print(model.get_feature_importance(Pool(train_data, label_values)))

Accuracy: 1.0
[1.19219061e-03 1.26397018e-04 2.78429900e+01 2.78803703e+01
 4.42744848e+01 8.36384143e-04]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [28]:
X, y_age, y_sex, features = read_dataset("src/resources/vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [30]:
model = CatBoostClassifier(
    loss_function='MultiClass'
)
model.fit(
    X=X_train,
    y=y_age_train, 
    verbose=False,
    eval_set=(X_eval, y_age_eval)
)
print("Accuracy:", np.mean(model.predict(X_test).reshape(-1) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(Pool(X_train, y_age_train)), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7389659520807061
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. leprum
5. rhymes
6. dayvinchik
7. tumblr_vacuum
8. xfilm
9. mudakoff
10. rapnewrap


#### Пол

In [31]:
model = CatBoostClassifier(
    loss_function='MultiClass'
)
model.fit(
    X=X_train,
    y=y_sex_train, 
    verbose=False,
    eval_set=(X_eval, y_sex_eval)
)
print("Accuracy:", np.mean(model.predict(X_test).reshape(-1) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(Pool(X_train, y_sex_train)), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8474148802017655
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. academyofman
5. 9o_6o_9o
6. be.beauty
7. femalemem
8. thesmolny
9. cook_good
10. i_d_t
